In [1]:
# %pip install --upgrade pip
# %pip install --disable-pip-version-check \
#     torch==1.13.1 \
#     torchdata==0.5.1 --quiet

# %pip install \
#     transformers==4.27.2 \
#     datasets==2.11.0 \
#     evaluate==0.4.0 \
#     rouge_score==0.1.2 \
#     peft==0.3.0 --quiet

# Installing the Reinforcement Learning library directly from github.
%pip install git+https://github.com/lvwerra/trl.git@25fa1bd    

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/lvwerra/trl.git (to revision 25fa1bd) to /tmp/pip-req-build-8y3fp86x
  Running command git clone --filter=blob:none --quiet https://github.com/lvwerra/trl.git /tmp/pip-req-build-8y3fp86x
  Running command git checkout -q 25fa1bd
  Resolved https://github.com/lvwerra/trl.git to commit 25fa1bd
  Preparing metadata (setup.py) ... done
  Created wheel for trl: filename=trl-0.4.2.dev0-py3-none-any.whl size=67540 sha256=bb3c2014e39239efda0ec4e087def2c28706c18a9a50c21bb585a37cee47d306
  Stored in directory: /tmp/pip-ephem-wheel-cache-hez8ozun/wheels/83/c2/4f/abce0f74327d238d71d1fdaa7a23c24cf393212007035862ce
Successfully built trl

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Import the necessary components. Some of them are new for this week, they will be discussed later in the notebook. 

In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig
from datasets import load_dataset
from peft import PeftModel, PeftConfig, LoraConfig, TaskType

# trl: Transformer Reinforcement Learning library
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

import torch
import evaluate

import numpy as np
import pandas as pd

# tqdm library makes the loops show a smart progress meter.
from tqdm import tqdm
tqdm.pandas()

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-08-23 13:27:53.812611: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 13:27:55.202676: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-23 13:27:55.202785: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.s


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)


In [2]:
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "3"  
torch.cuda.device_count()

4

# Load Model

In [3]:
class PeftModel:
    @staticmethod
    def load_base_model(model_path="google/flan-t5-base"):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = AutoModelForSeq2SeqLM.from_pretrained(
            model_path, torch_dtype=torch.bfloat16
        ).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        return model, tokenizer

    @staticmethod
    def load_from_peft_adapter(
        base_model_path, peft_model_path, train=False, merge_adapter=True
    ):
        model, tokenizer = self.load_base_model(base_model_path)
        
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = PeftModel.from_pretrained(
            model, peft_model_path, torch_dtype=torch.bfloat16, is_trainable=train
        ).to(device)

        if merge_adapter:
            model = model.merge_and_unload()

            if train:
                for param in model.parameters():
                    param.requires_grad = True

        # merge the adapter to the main model
        return model, tokenizer

    @staticmethod
    def save_peft_adapter(model, tokenizer, model_path):
        model.save_pretrained(model_path)
        tokenizer.save_pretrained(model_path)

    @staticmethod
    def merge_peft_and_save(model, tokenizer, model_path):
        model = model.merge_and_unload()
        model.save_pretrained(model_path)
        tokenizer.save_pretrained(model_path)
        
        
peft_model, tokenizer = PeftModel.load_base_model("./checkpoint/")

<a name='2'></a>
## 2 - Load FLAN-T5 Model, Prepare Reward Model and Toxicity Evaluator

<a name='2.1'></a>
### 2.1 - Load Data and FLAN-T5 Model Fine-Tuned with Summarization Instruction

You will keep working with the same Hugging Face dataset [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) and the pre-trained model [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5). 

In [4]:
import pickle
import re
from tqdm import tqdm 

save_pkl = lambda data, filepath: pickle.dump(data, open(filepath, "wb"))
load_pkl = lambda filepath: pickle.load(open(filepath, "rb"))

training_data = load_pkl("./rank_data/data.pkl")
training_data[-4]

('Summarize this news article in 50 words.\n\nNandan Nilekani Infosys Chairman Nandan Nilekani said on June 22 that e-commerce companies will need Open Network for Digital Commerce (ONDC) for online purchases to grow from the current 80 million monthly transacting users (MTUs) to 500 million. "ONDC will create a new class of e-commerce users. Suddenly, people who didn\'t think of e-commerce earlier will start thinking of it for convenience and order from their neighbourhood stores via ONDC," he said. "The number of platforms which are in e-commerce will go up. It won\'t be just two guys, but 10 guys. It might not benefit one guy, but it will benefit India," he added. Nilekani, who has been instrumental in creating India\'s digital public infrastructure from Aadhaar and UPI to Fastag and ONDC, was speaking at an event organised by management consulting firm Redseer. The Infosys veteran highlighted the rapid growth of digital public infrastructure in India, and how it has increased bank 

In [5]:
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.tokenizer = tokenizer
        self.data = data
        self.max_len = tokenizer.model_max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # self.data[index] = [prompt, human text, model text]
        prompt, summary, _ = self.data[index]
        inputs = self.tokenizer(prompt)

        input_ids = inputs['input_ids']
        query = self.tokenizer.decode(input_ids)

        return {
            'prompt': prompt,
            'summary': summary,
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'query': query
        }
    
    
train_ratio = 0.8
train_size = int(len(training_data) * train_ratio)

train_data = training_data[:train_size].copy()
test_data = training_data[train_size:].copy()

training_set = CustomDataset(train_data, tokenizer)
testing_set = CustomDataset(test_data, tokenizer)

training_set[0]

Token indices sequence length is longer than the specified maximum sequence length for this model (724 > 512). Running this sequence through the model will result in indexing errors


{'prompt': "Summarize this news article in 50 words.\n\nMy teachers have taught me that patience and perseverance always do wonders in everyone’s life, says Sonu Sood My teachers have taught me that patience and perseverance always do wonders in everyone’s life, says Sonu Sood He exclusively spoke to us about the importance of Teacher`s Day, amongst other things Official Instagram Account of Sonu Sood The pandemic saw the reel-life actor Sonu Sood turning a real-life hero. Besides facilitating the travel of the needy to meet their loved ones, providing free education, medical help and much moreâ\x80¦ Sood lent a helping hand to many. He was even nicknamed as â\x80\x98Messiah' in lieu of the help which he rendered during the pandemic. Mid-Day Online caught up with Sonu Sood for an exclusive interview on Teacher's Day. Since Sonu's mother was a teacher herself, Sonu spoke about the importance of the day, besides jogging down memory lane. Who was your favourite teacher in school or colleg

In [6]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"\ntrainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print_number_of_trainable_model_parameters(peft_model)

'\ntrainable model parameters: 247577856\nall model parameters: 247577856\npercentage of trainable model parameters: 100.00%'

Add the adapter to the original FLAN-T5 model. In the previous lab you were adding the fully trained adapter only for inferences, so there was no need to pass LoRA configurations doing that. Now you need to pass them to the constructed PEFT model, also putting `is_trainable=True`.

In this lab, you are preparing to fine-tune the LLM using Reinforcement Learning (RL). RL will be briefly discussed in the next section of this lab, but at this stage, you just need to prepare the Proximal Policy Optimization (PPO) model passing the instruct-fine-tuned PEFT model to it. PPO will be used to optimize the RL policy against the reward model.

In [7]:
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(peft_model,                                                               
                                                               torch_dtype=torch.bfloat16,
                                                               is_trainable=True)

print(f'PPO model parameters to be updated (ValueHead + 769 params):\n{print_number_of_trainable_model_parameters(ppo_model)}\n')
print(ppo_model.v_head)

PPO model parameters to be updated (ValueHead + 769 params):

trainable model parameters: 247578625
all model parameters: 247578625
percentage of trainable model parameters: 100.00%

ValueHead(
  (dropout): Dropout(p=0.1, inplace=False)
  (summary): Linear(in_features=768, out_features=1, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


During PPO, only a few parameters will be updated. Specifically, the parameters of the `ValueHead`. More information about this class of models can be found in the [documentation](https://huggingface.co/docs/trl/main/en/models#trl.create_reference_model). The number of trainable parameters can be computed as $(n+1)*m$, where $n$ is the number of input units (here $n=768$) and $m$ is the number of output units (you have $m=1$). The $+1$ term in the equation takes into account the bias term.

Now create a frozen copy of the PPO which will not be fine-tuned - a reference model. The reference model will represent the LLM before detoxification. None of the parameters of the reference model will be updated during PPO training. This is on purpose.

In [8]:
ref_model = create_reference_model(ppo_model)

print(f'Reference model parameters to be updated:\n{print_number_of_trainable_model_parameters(ref_model)}\n')

Reference model parameters to be updated:

trainable model parameters: 0
all model parameters: 247578625
percentage of trainable model parameters: 0.00%



Everything is set. It is time to prepare the reward model!

<a name='2.2'></a>
### 2.2 - Prepare Reward Model

**Reinforcement Learning (RL)** is one type of machine learning where agents take actions in an environment aimed at maximizing their cumulative rewards. The agent's behavior is defined by the **policy**. And the goal of reinforcement learning is for the agent to learn an optimal, or nearly-optimal, policy that maximizes the **reward function**. 

In [9]:
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config

In [10]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.tokenizer = tokenizer
        self.data = data
        self.max_len = tokenizer.model_max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # [prompt, human text, model text]
        inputs = self.tokenizer(self.data[index],             
                                add_special_tokens=True,
                                max_length=self.max_len,
                                pad_to_max_length=True,
                                return_token_type_ids=True,
                                truncation=True,
                                padding="max_length")

        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        }

In [11]:
import torch.nn as nn


class RewardModel(torch.nn.Module):
    def __init__(self, dropout=0.3):
        super(RewardModel, self).__init__()
        self.l1 = GPT2Model.from_pretrained("gpt2")
        
        for param in self.l1.parameters():
            param.requires_grad = False
        
        self.l2 = nn.Sequential(
            nn.Linear(768, 1),
        )
        self.outl = nn.Sigmoid()
        
    def gpt2(self, ids, attention_mask, token_type_ids):
        # logits shape: [batch=16, seqlen=1024, dim=768]
        logits, _ = self.l1(ids, 
                         attention_mask=attention_mask,
                         token_type_ids=token_type_ids,
                         return_dict=False)
        
        sequence_lengths = (torch.eq(ids[:, 0], tokenizer.pad_token_id).long().argmax(-1) - 1).to(
                    logits.device
                )
        
        batch_size = ids.shape[0]
        
        # pooled logits shape: [batch=16, dim=768]
        pooled_logits = logits[torch.arange(batch_size, device=logits.device), sequence_lengths]
        return pooled_logits
    
    def forward(self, ids, mask, token_type_ids):
        """
        ids shape = [batch, 3, 512]
        3: prompt, human_text, model_text
        """  
        prompt = self.gpt2(ids[:, 0], 
                         attention_mask=mask[:, 0], 
                         token_type_ids=token_type_ids[:, 0])
        
        human_text = self.gpt2(ids[:, 1], 
                             attention_mask=mask[:, 1], 
                             token_type_ids=token_type_ids[:, 1])
            
        model_text = self.gpt2(ids[:, 2], 
                             attention_mask=mask[:, 2], 
                             token_type_ids=token_type_ids[:, 2])

        human_score = self.l2(prompt + human_text)
        model_score = self.l2(prompt + model_text)
        
        return self.outl(human_score - model_score)
    
    def predict(self, ids, mask, token_type_ids):
        """
        ids shape: [batch, 2, 512]
        2: prompt, text
        """
        prompt = self.gpt2(ids[:, 0], 
                         attention_mask=mask[:, 0], 
                         token_type_ids=token_type_ids[:, 0])
        
        text = self.gpt2(ids[:, 1], 
                       attention_mask=mask[:, 1], 
                       token_type_ids=token_type_ids[:, 1])
        
        return self.l2(prompt + text)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
reward_model = RewardModel().to(device)

model_path = "./reward_model_checkpoint/gpt2_last_layer_trained_model.pt"

if os.path.isfile(model_path):
    reward_model.load_state_dict(
        torch.load(model_path, map_location=device), strict=False
    )


reward_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
reward_tokenizer.pad_token = reward_tokenizer.eos_token

def reward(inputs):
    """
    inputs = [[prompt, summary], [prompt, summary]]
    """
    reward_model.eval()
    
    dataset = CustomDataset(inputs, reward_tokenizer)
    params = {'batch_size': len(inputs),
              'shuffle': False,
              'num_workers': 0}
    dataloader = DataLoader(dataset, **params)
    
    with torch.no_grad():
        reward = []
        
        for idx, data in enumerate(dataloader):
            ids = data["ids"].to(device)
            mask = data["mask"].to(device)
            token_type_ids = data["token_type_ids"].to(device)

            pred = reward_model.predict(ids, mask, token_type_ids)
            reward.append(pred.squeeze(-1))
            
        return [tensor for tensor in torch.cat(reward, dim=-1)]

In [13]:
reward([["kjhgf hgf", "gtreef regef"],["54y435tgr", "regtw5exqwe"]])

[tensor(9.2640, device='cuda:0'), tensor(8.7152, device='cuda:0')]

Take some non-toxic text, tokenize it, and pass it to the model. Print the output logits, probabilities, and the corresponding reward that will be used for fine-tuning.

<a name='3.1'></a>
### 3.1 - Initialize `PPOTrainer`
 
For the `PPOTrainer` initialization, you will need a collator. Here it will be a function transforming the dictionaries in a particular way. You can define and test it:

In [14]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

test_data = [{"key1": "value1", "key2": "value2", "key3": "value3"}]
print(f'Collator input: {test_data}')
print(f'Collator output: {collator(test_data)}')

Collator input: [{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}]
Collator output: {'key1': ['value1'], 'key2': ['value2'], 'key3': ['value3']}


Set up the configuration parameters. Load the `ppo_model` and the tokenizer. You will also load a frozen version of the model `ref_model`. The first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This works as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original LLM.

In [15]:
learning_rate=1.41e-5
max_ppo_epochs=1
mini_batch_size=2
batch_size=4

config = PPOConfig(
    # model_name=model_name,    
    learning_rate=learning_rate,
    ppo_epochs=max_ppo_epochs,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size
)

ppo_trainer = PPOTrainer(config=config, 
                         model=ppo_model, 
                         ref_model=ref_model, 
                         tokenizer=tokenizer, 
                         dataset=training_set, 
                         data_collator=collator)

<a name='3.2'></a>
### 3.2 - Fine-Tune the Model

The fine-tuning loop consists of the following main steps:
1. Get the query responses from the policy LLM (PEFT model).
2. Get sentiments for query/responses from hate speech RoBERTa model.
3. Optimize policy with PPO using the (query, response, reward) triplet.

The operation is running if you see the following metrics appearing:
* `objective/kl`: minimize kl divergence,
* `ppo/returns/mean`: maximize mean returns,
* `ppo/policy/advantages_mean`: maximize advantages.

In [16]:
output_min_length = 200
output_max_length = 400
not_hate_index = 0
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    # "min_length": 5,
    # "top_k": 0.0,
    # "top_p": 1.0,
    # "do_sample": True
    "max_new_tokens": 200
}

reward_kwargs = {
    "top_k": None, # Return all scores.
    "function_to_apply": "none", # You want the raw logits without softmax.
    "batch_size": 16
}

# max_ppo_steps = 10

for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    # Break when you reach max_steps.
    # if step >= max_ppo_steps:
    #     break   

    # prompt tensors is a list of 1D tensors
    prompt_tensors = batch["input_ids"]
    # print(prompt_tensors[0].shape, "prompt tensor shape")

    # Get response from FLAN-T5/PEFT LLM.
    summary_tensors = []

    for idx, prompt_tensor in enumerate(prompt_tensors):
        max_new_tokens = output_length_sampler()        
            
        generation_kwargs["max_new_tokens"] = max_new_tokens
        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)
        
        # if idx == 0:
        #     print(max_new_tokens, "max new tokens")
        #     print(summary.shape, "summary shape")
        #     print(summary.squeeze()[-max_new_tokens:].shape, "summary shape post processing")
        #     print(summary.squeeze()[-max_new_tokens:].squeeze().shape, "summary shape post processing 2")
        #     print(summary.squeeze()[-max_new_tokens:].squeeze())
        
        summary_tensors.append(summary.squeeze()[-max_new_tokens:])
        
    # This needs to be called "response".
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in summary_tensors]

    # Compute reward outputs.
    query_response_pairs = list(zip(batch["query"], batch["response"]))
    reward_tensors = reward(query_response_pairs)

    # Run PPO step.
    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)
    
    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-'.join('' for x in range(100)))

0it [00:00, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
1it [00:14, 14.76s/it]

objective/kl: 0.0
ppo/returns/mean: 0.8067174553871155
ppo/policy/advantages_mean: 3.448037233511059e-08
---------------------------------------------------------------------------------------------------


2it [00:19,  9.09s/it]

objective/kl: -0.011627238243818283
ppo/returns/mean: 3.6164681911468506
ppo/policy/advantages_mean: 4.9511168498383995e-08
---------------------------------------------------------------------------------------------------


3it [00:29,  9.33s/it]

...

objective/kl: -0.01963803358376026
ppo/returns/mean: 2.0179033279418945
ppo/policy/advantages_mean: -3.3200393545484985e-08
---------------------------------------------------------------------------------------------------


16it [01:44,  5.88s/it]

objective/kl: 0.012761603109538555
ppo/returns/mean: 3.654768228530884
ppo/policy/advantages_mean: 2.5981509566008754e-07
---------------------------------------------------------------------------------------------------


16it [01:51,  6.98s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1438028/1419551812.py:57 in <cell line: 22>                                       │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1438028/1419551812.py'                      │
│                                                                                                  │
│ /usr/lib/python3.8/contextlib.py:75 in inner                                                     │
│                                                                                                  │
│    72 │   │   @wraps(func)                                                                       │
│    73 │   │   def inner(*args, **kwds):                                                          │
│    74 │   │   │   with self._recreate_cm():                                                      │
│ ❱  75 │   │   │   │   return func(*args, **kwds)                                                 │
│    76 │   │   return inner                                                                       │
│    77                                                                                            │
│    78                                                                                            │
│                                                                                                  │
│ /home/qblocks/.local/lib/python3.8/site-packages/trl/trainer/ppo_trainer.py:666 in step          │
│                                                                                                  │
│    663 │   │   │   │   │   │   self.model, batch["queries"], batch["responses"], model_inputs,   │
│    664 │   │   │   │   │   )                                                                     │
│    665 │   │   │   │                                                                             │
│ ❱  666 │   │   │   │   train_stats = self.train_minibatch(                                       │
│    667 │   │   │   │   │   batch["logprobs"],                                                    │
│    668 │   │   │   │   │   batch["values"],                                                      │
│    669 │   │   │   │   │   batch["rewards"],                                                     │
│                                                                                                  │
│ /usr/lib/python3.8/contextlib.py:75 in inner                                                     │
│                                                                                                  │
│    72 │   │   @wraps(func)                                                                       │
│    73 │   │   def inner(*args, **kwds):                                                          │
│    74 │   │   │   with self._recreate_cm():                                                      │
│ ❱  75 │   │   │   │   return func(*args, **kwds)                                                 │
│    76 │   │   return inner                                                                       │
│    77                                                                                            │
│    78                                                                                            │
│                                                                                                  │
│ /home/qblocks/.local/lib/python3.8/site-packages/trl/trainer/ppo_trainer.py:927 in               │
│ train_minibatch                                                                                  │
│                                                                                                  │
│    924 │   │   loss_p, loss_v, train_stats = self.loss(old_logprobs, values, rewards, logits, v  │
│    925 │   │   loss = loss_p + loss_v                      

<a name='3.4'></a>
### 3.4 - Evaluate the Model Qualitatively

Let's inspect some examples from the test dataset. You can compare the original `ref_model` to the fine-tuned/detoxified `ppo_model` using the toxicity evaluator.

In [17]:
params = {'batch_size': 8,
               'shuffle': False,
               'num_workers': 0
                }

loader = DataLoader(testing_set, **params)

In [18]:
batch_size = 1
compare_results = {}

# df_batch = next(iter(loader))
df_batch = collator([testing_set[51]])

compare_results["query"] = df_batch["query"]
compare_results["human_summary"] = df_batch["summary"]

prompt_tensors = df_batch["input_ids"]

summary_tensors_ref = []
summary_tensors = []

# Get response from ppo and base model.
for i in tqdm(range(batch_size)):
    gen_len = output_length_sampler()
    generation_kwargs["max_new_tokens"] = gen_len
    
    summary = ref_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device), 
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors_ref.append(summary)

    summary = ppo_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device), 
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors.append(summary)

# Decode responses.
compare_results["response_before"] = [tokenizer.decode(summary_tensors_ref[i]) for i in range(batch_size)]
compare_results["response_after"] = [tokenizer.decode(summary_tensors[i]) for i in range(batch_size)]

# Sentiment analysis of query/response pairs before/after.
texts_before = list(zip(compare_results["query"], compare_results["response_before"]))
rewards_before = reward(texts_before)
compare_results["reward_before"] = rewards_before

texts_after = list(zip(compare_results["query"], compare_results["response_after"]))
rewards_after = reward(texts_after)
compare_results["reward_after"] = rewards_after

100%|██████████| 1/1 [00:00<00:00,  1.12it/s]


In [19]:
compare_results

{'query': ['Give a title to the given news article in not more than 10 words. Mumbai-based automobile classifieds company CarTrade has converted itself from a private limited company to a public limited company via special resolution ahead of its initial public offering (IPO) this year, according to the company\'s filings with the Registrar of Companies (ROC) at the Ministry of Corporate Affairs (MCA). Special Resolution allows provisions under Companies Act 2013 to alter its conditions such as conversion of a Private Company into a Public Company. "The name was changed and conversion from Private to Public vide Special Resolution passed by the Members at the Extraordinary General Meeting of the Company held on April 29, 2021," read the filings. The company’s registered name is now CarTrade Tech Limited. The documents also stated that CarTrade\'s co-founder, Vinay Vinod Sanghi, will take position as Managing Director effective from March 31, 2021 till March 30, 2026. Started in 2010, C

In [42]:
compare_results

{'query': ["Summarize this news article in 50 words. Arc Ventures has tied up with Japan-based Leave A Nest to set up ArcNest It includes an initial INR 50 Cr fund for ideas that are combating India's environmental concerns India’s rising pollution stats have opened up more opportunities for cleantech startups to succeed The world has become hotter in the last five years and so have cleantech startups in India. Be it monitoring air quality or solving complex issues like stubble burning with innovative ideas, cleantech startups, backed by the sharpest minds from IITs and IIMs, bet high on making a difference while running a sustainable business. With shocking reports coming in about the impact of pollution — like the centre for science and environment (CSE)’s report in June this year, which said life expectancy in India is going down by 2.6 years due to air pollution —- innovative ideas from across the country are being appreciated by government, masses and investors alike. “Our country

In [38]:
compare_resultse_results

{'query': ["Summarize this news article in 50 words. Arc Ventures has tied up with Japan-based Leave A Nest to set up ArcNest It includes an initial INR 50 Cr fund for ideas that are combating India's environmental concerns India’s rising pollution stats have opened up more opportunities for cleantech startups to succeed The world has become hotter in the last five years and so have cleantech startups in India. Be it monitoring air quality or solving complex issues like stubble burning with innovative ideas, cleantech startups, backed by the sharpest minds from IITs and IIMs, bet high on making a difference while running a sustainable business. With shocking reports coming in about the impact of pollution — like the centre for science and environment (CSE)’s report in June this year, which said life expectancy in India is going down by 2.6 years due to air pollution —- innovative ideas from across the country are being appreciated by government, masses and investors alike. “Our country